In [3]:
from urllib.parse import urlparse
from re import compile
import ipaddress
import re
import whois
from datetime import datetime,timezone
from bs4 import BeautifulSoup
import urllib
import urllib.request
import lxml

In [7]:
class LexicalFeatures:
    def __init__(self,url):
        self.description = 'URL'
        self.url = url
        self.urlparse = urlparse(self.url)
        self.host = self.urlparse.hostname
        self.today = datetime.now().replace(tzinfo=None)
        try:
            self.whois=whois.whois(self.urlparse.netloc)
        except:
            self.whois=None
    
    def url_length(self):
        return len(self.url)
    
    def url_scheme(self):
        return self.urlparse.scheme
    def url_path_length(self):
        return len(self.urlparse.path)
    def url_host_length(self):
        return len(self.host)
    def url_ip_address(self):
        try:
            ipaddress.ip_address(self.urlparse.netloc)
            ip=1
        except ValueError:
            ip=0
        return ip
    def url_number_of_digits(self):
        digits = [i for i in self.url if i.isdigit()]
        return len(digits)
    def url_number_of_parameters(self):
        params = self.urlparse.query
        return 0 if params == '' else len(params.split('&'))
    def url_number_of_fragments(self):
        frags = self.urlparse.fragment
        return len(frags.split('#')) if frags!='' else 0

    def is_url_encoded(self):
        return '%' in self.url.lower()

    def url_number_encoded_char(self):
        encs = [i for i in self.url if i == '%']
        return len(encs)

#     def url_string_entropy(self):
#         return self.__get_entropy(self.url)

    def url_number_of_subdirectories(self):
        d = self.urlparse.path.split('/')
        return len(d)-1

    def url_number_of_periods(self):
        periods = [i for i in self.url if i == '.']
        return len(periods)

    def url_has_client_in_string(self):
        return 'client' in self.url.lower()

    def url_has_admin_in_string(self):
        return 'admin' in self.url.lower()

    def url_has_server_in_string(self):
        return 'server' in self.url.lower()

    def url_has_login_in_string(self):
        return 'login' in self.url.lower()
        
    def url_get_tld(self):
          return self.urlparse.netloc.split('.')[-1].split(':')[0]
    def url_number_of_subdomains(self):
        subdomains = len(self.urlparse.netloc.split(':')[0].split('.'))-1
        return subdomains if subdomains>=0 else 0 
    def url_number_of_dashes(self):
        return 1 if len([i for i in self.urlparse.netloc if i=='-'])>0 else 0 
    def redirection(self):
        pos = self.url.rfind('//')
        if pos > 6:
            if pos > 7:
                return 1
            else:
                return 0
        else:
            return 0
    def url_have_at_sign(self):
          return 1 if '@' in self.url else 0
    def is_tiny_url(self):
        #listing shortening services
            shortening_services = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                      r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                      r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                      r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                      r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                      r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                      r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                      r"tr\.im|link\.zip\.net"
            match=re.search(shortening_services,self.url)
            if match:
                return 1
            else:
                return 0
    def daysSinceRegistration(self):
        if self.whois and self.whois['creation_date']:
            diff = self.today - self.whois['creation_date'].replace(tzinfo=None)
            diff = str(diff).split(' days')[0]
            return diff
        else:
            return 0

    def daysSinceExpiration(self):
        if self.whois and self.whois['expiration_date']:
            diff = self.whois['expiration_date'].replace(tzinfo=None) - self.today
            diff = str(diff).split(' days')[0]
            return diff
        else:
            return 0
        
    def url_number_of_uppercase(self):
        return len([i for i in self.url if i.isupper()])
    
    def url_number_of_spaces(self):
        return self.url.count("%20")
    def url_number_of_zeroes(self):
        return self.url.count('0')
    def is_url_single_character_directory(self):
        for i in self.urlparse.path.split('/'):
            if len(i)==1:
                return 1
        return 0
    def url_ratio_upper_to_lower(self):
        l2=len([i for i in self.url if i.islower()])
        return len([i for i in self.url if i.isupper()])/(l2 if l2>0 else 1000000)
    def is_url_in_dns_record(self):
        return 1 if not self.whois.registrar else 0

    def url_web_traffic(self):
        try:
            url = urllib.parse.quote(self.url)
            rank = BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + url).read(), "xml").find(
                "REACH")['RANK']
            rank = int(rank)
            return rank
        except TypeError:
                return 0

l = LexicalFeatures('https://www.geeksforgeeks.org/ternary-operator-in-python/');
l.url_scheme()
l.url_ip_address()
l.url_number_of_digits()
l.url_number_of_parameters()
l.url_number_of_fragments()
l.is_url_encoded()
l.url_number_encoded_char()
l.url_number_of_subdirectories()
l.url_number_of_periods()
l.url_has_client_in_string()
l.url_has_server_in_string()
l.url_has_server_in_string()
l.url_has_login_in_string()
l.url_get_tld()
l.url_number_of_subdomains()
l.url_number_of_dashes()
l.redirection()
l.url_have_at_sign()
l.is_tiny_url()
l.daysSinceRegistration()
l.daysSinceExpiration()
l.url_number_of_uppercase()
l.url_number_of_spaces()
l.url_number_of_zeroes()
l.is_url_single_character_directory()
l.url_ratio_upper_to_lower()
l.is_url_in_dns_record()
l.url_web_traffic()

163

In [143]:
l.whois

{'domain_name': 'GEEKSFORGEEKS.ORG',
 'registrar': 'PDR Ltd. d/b/a PublicDomainRegistry.com',
 'whois_server': 'whois.publicdomainregistry.com',
 'referral_url': None,
 'updated_date': [datetime.datetime(2022, 4, 21, 6, 36, 7),
  datetime.datetime(2022, 4, 21, 6, 36, 8)],
 'creation_date': datetime.datetime(2009, 3, 19, 6, 8, 55),
 'expiration_date': datetime.datetime(2030, 3, 19, 6, 8, 55),
 'name_servers': ['NS-1520.AWSDNS-62.ORG',
  'NS-1569.AWSDNS-04.CO.UK',
  'NS-245.AWSDNS-30.COM',
  'NS-869.AWSDNS-44.NET',
  'ns-1520.awsdns-62.org',
  'ns-1569.awsdns-04.co.uk',
  'ns-245.awsdns-30.com',
  'ns-869.awsdns-44.net'],
 'status': 'clientTransferProhibited https://icann.org/epp#clientTransferProhibited',
 'emails': ['abuse@publicdomainregistry.com',
  'contact@privacyprotect.org',
  'abuse-contact@publicdomainregistry.com'],
 'dnssec': ['unsigned', 'Unsigned'],
 'name': 'Domain Admin',
 'org': 'Privacy Protect, LLC (PrivacyProtect.org)',
 'address': '10 Corporate Drive',
 'city': 'Burl

In [5]:
# BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + l.url).read(),"lxml")

C:\Users\princ\AppData\Local\Programs\Python\Python310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


<?xml version="1.0" encoding="UTF-8"?><!-- Need more Alexa data?  Find our APIs here: https://aws.amazon.com/marketplace/seller-profile?id=4a9dbf38-88b1-4e87-a459-271154a77d2e --><html><body><alexa aid="=" home="0" idn="geeksforgeeks.org/ternary-operator-in-python" url="geeksforgeeks.org/ternary-operator-in-python" ver="0.9">
<sd flags="" host="geeksforgeeks.org" title="A">
<title text="GeeksforGeeks"></title>
</sd>
<sd><popularity source="panel" text="205" url="geeksforgeeks.org/"></popularity><reach rank="163"></reach><rank delta="-115"></rank><country code="IN" name="India" rank="58"></country></sd></alexa></body></html>